In [ ]:
import torch
from torch_geometric.data import Data

def build_similarity_graph(features, threshold=0.75):
    """Step 1: Graph Construction 基于余弦相似度 [cite: 101, 108, 111]"""
    # 归一化特征
    features_norm = F.normalize(features, p=2, dim=1)
    # 计算全矩阵相似度 [cite: 109]
    sim_matrix = torch.mm(features_norm, features_norm.t())
    
    # 根据阈值建立边 s_ij > tau [cite: 111, 298]
    edge_index = (sim_matrix > threshold).nonzero(as_tuple=False).t()
    
    # 移除自环
    edge_index = edge_index[:, edge_index[0] != edge_index[1]]
    return edge_index

# 示例：将您的数据打包成 PyG 格式
def prepare_data(image_embeddings, labels):
    edge_index = build_similarity_graph(image_embeddings)
    # 预先计算拓扑特征作为模型输入
    f_u = extract_topological_features(edge_index, image_embeddings.size(0), labels)
    return Data(x=image_embeddings, edge_index=edge_index, f_u=f_u, y=labels)